# Задание 4. Реализовать рекомендательную систему по подбору пива на основе  датасета «BeerAdvocate»

## Downloading and unzipping data

In [ ]:
# Change directory for kaggle JSON
import os
import warnings

os.chdir("/Users/pan_ae/Study/ml academy/task_4/task_4")

In [4]:
# Create a kaggle folder
!mkdir -p ~/.kaggle

# Copy kaggle.json to created folder
!cp kaggle.json ~/.kaggle/

In [ ]:
os.chdir("/content")

In [5]:
# Permission for the json to act
!chmod 600 ~/.kaggle/kaggle.json

In [10]:
# Download the required dataset
!kaggle datasets download -d thedevastator/1-5-million-beer-reviews-from-beer-advocate

 99%|█████████████████████████████████████▍| 32.0M/32.5M [00:05<00:00, 7.66MB/s]
100%|██████████████████████████████████████| 32.5M/32.5M [00:05<00:00, 6.17MB/s]


In [11]:
# Unzip our dataset
from zipfile import ZipFile
from tqdm import tqdm


file_to_extract = "1-5-million-beer-reviews-from-beer-advocate.zip"

# Open your .zip file
with ZipFile(file=file_to_extract) as zip_file:

    # Loop over each file and extract them
    for file in tqdm(iterable=zip_file.namelist(), total=len(zip_file.namelist())):
        zip_file.extract(member=file)

100%|██████████| 1/1 [00:00<00:00,  1.23it/s]


## Data Preprocessing

In [130]:
import pandas as pd
import numpy as np
import time
import warnings
warnings.filterwarnings("ignore")

In [5]:
start_time = time.time()
df = pd.read_csv("beer_reviews.csv")
end_time = time.time()

print(f"Elapsed time to read csv-file is: {end_time - start_time}")
df.head(10)

Elapsed time to read csv-file is: 3.8526830673217773


,index,brewery_id,brewery_name,review_time,review_overall,review_aroma,review_appearance,review_profilename,beer_style,review_palate,review_taste,beer_name,beer_abv,beer_beerid
0,0,10325,Vecchio Birraio,1234817823,1.5,2.0,2.5,stcules,Hefeweizen,1.5,1.5,Sausa Weizen,5.0,47986
1,1,10325,Vecchio Birraio,1235915097,3.0,2.5,3.0,stcules,English Strong Ale,3.0,3.0,Red Moon,6.2,48213
2,2,10325,Vecchio Birraio,1235916604,3.0,2.5,3.0,stcules,Foreign / Export Stout,3.0,3.0,Black Horse Black Beer,6.5,48215
3,3,10325,Vecchio Birraio,1234725145,3.0,3.0,3.5,stcules,German Pilsener,2.5,3.0,Sausa Pils,5.0,47969
4,4,1075,Caldera Brewing Company,1293735206,4.0,4.5,4.0,johnmichaelsen,American Double / Imperial IPA,4.0,4.5,Cauldron DIPA,7.7,64883
5,5,1075,Caldera Brewing Company,1325524659,3.0,3.5,3.5,oline73,Herbed / Spiced Beer,3.0,3.5,Caldera Ginger Beer,4.7,52159
6,6,1075,Caldera Brewing Company,1318991115,3.5,3.5,3.5,Reidrover,Herbed / Spiced Beer,4.0,4.0,Caldera Ginger Beer,4.7,52159
7,7,1075,Caldera Brewing Company,1306276018,3.0,2.5,3.5,alpinebryant,Herbed / Spiced Beer,2.0,3.5,Caldera Ginger Beer,4.7,52159
8,8,1075,Caldera Brewing Company,1290454503,4.0,3.0,3.5,LordAdmNelson,Herbed / Spiced Beer,3.5,4.0,Caldera Ginger Beer,4.7,52159
9,9,1075,Caldera Brewing Company,1285632924,4.5,3.5,5.0,augustgarage,Herbed / Spiced Beer,4.0,4.0,Caldera Ginger Beer,4.7,52159


Let's look at the [dataset's](https://www.kaggle.com/datasets/thedevastator/1-5-million-beer-reviews-from-beer-advocate/data) columns description from Kaggle 

|Column name| Description                                         |
|---|---------------|
|brewery_name| The name of the brewery that made the beer. (String) |
|review_time|The date and time of the review. (String)|
|review_overall|The reviewer's overall rating of the beer on a scale of 1 to 5. (Float)|
|review_aroma|The reviewer's rating of the beer's aroma on a scale of 1 to 5. (Float)|
|review_appearance|The reviewer's rating of the beer's appearance on a scale of 1 to 5. (Float)|
|review_profilename|The reviewer's username. (String)|
|beer_style|The style of beer. (String)|
|review_palate|The reviewer's rating of the beer's palate on a scale of 1 to 5. (Float)|
|review_taste|The reviewer's rating of the beer's taste on a scale of 1 to 5. (Float)|
|beer_name|The name of the beer. (String)|
|beer_abv|The alcohol by volume of the beer. (Float)|
|brewery_id||
|beer_beerid||

Checking for null values

In [6]:
df.isna().sum()

index                     0
brewery_id                0
brewery_name             15
review_time               0
review_overall            0
review_aroma              0
review_appearance         0
review_profilename      348
beer_style                0
review_palate             0
review_taste              0
beer_name                 0
beer_abv              67785
beer_beerid               0
dtype: int64

In [5]:
df.dropna(inplace=True)

Выберем основные признаки

In [15]:
df_1 = df[["beer_name", "review_profilename", "review_overall"]]
df_1.head()

,beer_name,review_profilename,review_overall
0,Sausa Weizen,stcules,1.5
1,Red Moon,stcules,3.0
2,Black Horse Black Beer,stcules,3.0
3,Sausa Pils,stcules,3.0
4,Cauldron DIPA,johnmichaelsen,4.0


In [16]:
df_1.drop_duplicates(inplace=True)

In [17]:
df_1.isna().sum()

beer_name               0
review_profilename    343
review_overall          0
dtype: int64

In [18]:
df_1.dropna(inplace=True)

## Modeling

### ItemKNN

ItemKNN, или item-based collaborative filtering, основан на идее о схожести между элементами (товарами, фильмами и т. д.) на основе истории предпочтений пользователей. Он работает следующим образом:

1. Сначала строится матрица взаимодействий пользователь-элемент, где каждая ячейка представляет собой оценку пользователя по отношению к элементу.
2. Затем вычисляется сходство между парами элементов на основе их взаимодействий с пользователями, часто с использованием меры сходства, такой как косинусное сходство или корреляция Пирсона.
3. Найти ближайших соседей для заданного элемента. Когда у вас есть матрица сходства, для введенного пива вы можете найти его ближайших соседей, то есть другие элементы, которые имеют наибольшее сходство с ним.
4. Составление рекомендаций. После того как найдены ближайшие соседи для введенного элемента, вы можете использовать предпочтения пользователей по этим соседям, чтобы сформировать список рекомендаций. Чем ближе элемент к введенному, тем выше его рекомендация.


Количество оценков для каждого продукта

In [136]:
beer_rating_count = (df_1.groupby(by=["beer_name"])["review_overall"].count().reset_index().rename(columns={"review_overall": "review_overall_count"}))

In [137]:
beer_rating_count.head()

,beer_name,review_overall_count
0,! (Old Ale),1
1,"""100""",5
2,"""100"" Pale Ale",1
3,"""12"" Belgian Golden Strong Ale",2
4,"""33"" Export",3


Присоединим полученный датафрейм к исходному

In [138]:
rating_with_review_overall_count = df_1.merge(beer_rating_count, on="beer_name", how="left")
rating_with_review_overall_count.head(10)

,beer_name,review_profilename,review_overall,review_overall_count
0,Sausa Weizen,stcules,1.5,1
1,Red Moon,stcules,3.0,1
2,Black Horse Black Beer,stcules,3.0,1
3,Sausa Pils,stcules,3.0,1
4,Cauldron DIPA,johnmichaelsen,4.0,1
5,Caldera Ginger Beer,oline73,3.0,9
6,Caldera Ginger Beer,Reidrover,3.5,9
7,Caldera Ginger Beer,alpinebryant,3.0,9
8,Caldera Ginger Beer,LordAdmNelson,4.0,9
9,Caldera Ginger Beer,augustgarage,4.5,9


Выберем только те пивные продукты, для которых имеем более 50 оценок

In [139]:
rating_popular_beer = rating_with_review_overall_count[rating_with_review_overall_count["review_overall_count"] > 50]
rating_popular_beer.shape

(1273781, 4)

In [140]:
# interaction_matrix = df_1.pivot_table(index='review_profilename', columns='beer_beerid', values='review_overall')
interaction_matrix = rating_popular_beer.pivot_table(index='beer_name', columns='review_profilename', values='review_overall').fillna(0)
interaction_matrix.head()

review_profilename,0110x011,01Ryan10,02maxima,03SVTCobra,04101Brewer,05Harley,0Naught0,0beerguy0,0runkp0s,0to15,...,zutmin,zwalk8,zwan,zwoehr,zymrgy,zymurgy4all,zymurgywhiz,zythus,zyzygy,zzajjber
beer_name,,,,,,,,,,,,,,,,,,,,,
"""Old Yeltsin"" Imperial Stout",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"""Shabadoo"" Black & Tan Ale",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
# 100,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
#9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
't Gaverhopke Extra,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [145]:
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
from sklearn.metrics import mean_squared_error, mean_absolute_error

In [123]:
beer_features_df_matrix = csr_matrix(interaction_matrix.values)
beer_features_df_matrix

<5090x30989 sparse matrix of type '<class 'numpy.float64'>'
	with 1260430 stored elements in Compressed Sparse Row format>

In [124]:
# Инициализируем модель ItemKNN
model_knn = NearestNeighbors(metric='cosine', algorithm='brute')
model_knn.fit(beer_features_df_matrix)

NearestNeighbors(algorithm='brute', metric='cosine')

In [125]:
beer = "Guinness Draught"

In [126]:
interaction_matrix[interaction_matrix.index.str.startswith("Guinness")]

review_profilename,0110x011,01Ryan10,02maxima,03SVTCobra,04101Brewer,05Harley,0Naught0,0beerguy0,0runkp0s,0to15,...,zutmin,zwalk8,zwan,zwoehr,zymrgy,zymurgy4all,zymurgywhiz,zythus,zyzygy,zzajjber
beer_name,,,,,,,,,,,,,,,,,,,,,
Guinness 250th Anniversary Stout,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Guinness Black Lager,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Guinness Draught,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,4.5,0.0,0.0,0.0,0.0
Guinness Draught Extra Cold,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Guinness Extra Stout (Original),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Guinness Foreign Extra Stout,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Guinness Special Export Stout (John Martin - Belgium),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [148]:
distances, indices = model_knn.kneighbors(interaction_matrix.loc[beer,:].values.reshape(1, -1), n_neighbors = 8)

In [149]:
# distances, indices = model_knn.kneighbors(interaction_matrix.loc[47986].values.reshape(1, -1), n_neighbors=5+1)

In [150]:
for i in range(0, len(distances.flatten())):
    if i == 0:
        print(f"Recommendations for {beer}:\n")
    else:
        print(f"{i}: {interaction_matrix.index[indices.flatten()[i]]}, with distance of {distances.flatten()[i]}")

Recommendations for Guinness Draught:

1: Samuel Adams Boston Lager, with distance of 0.5469162346698848
2: Sierra Nevada Pale Ale, with distance of 0.5901818923844364
3: Newcastle Brown Ale, with distance of 0.5936254018883957
4: Murphy's Irish Stout, with distance of 0.6027946209871407
5: Heineken Lager Beer, with distance of 0.6083413724606613
6: Samuel Adams Winter Lager, with distance of 0.6091611421829981
7: Bass Pale Ale, with distance of 0.6105648267027692


In [151]:
# Получить рейтинги этих соседей из interaction_matrix
neighbor_ratings = interaction_matrix.iloc[indices.flatten()]

# Усреднить рейтинги этих соседей
predicted_ratings = neighbor_ratings.mean(axis=0)

# Преобразуйте данные в numpy arrays, чтобы их можно было использовать с функцией mean_squared_error
actual_ratings = interaction_matrix.loc[beer,:].values

# Рассчитайте RMSE
rmse = np.sqrt(mean_squared_error(actual_ratings, predicted_ratings))
mae = mean_absolute_error(actual_ratings, predicted_ratings)
print("RMSE:", rmse, "\nMAE:", mae)

RMSE: 0.6818238594118137 
MAE: 0.22522676057095528


Сохраним модель

In [155]:
from joblib import dump, load

In [154]:
# Укажите путь и имя файла для сохранения модели
file_path = 'models/model_knn.joblib'

# Сохраните модель
dump(model_knn, file_path)

['models/model_knn.joblib']

In [156]:
# Загрузите модель
model_knn = load(file_path)